In [1]:
import tensorflow as tf

/home/trungdv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
saver = tf.train.import_meta_graph('../../../saved_models/attention_aps_word_context/csp.ckpt.meta')

var_map = {
    "decoder/decoder_emb_layer/kernel": "decoder/dense/kernel",
    "decoder/decoder_emb_layer/bias": "decoder/dense/bias",
    "decoder/decoder_emb_layer/bias/Adam": "decoder/dense/bias/Adam",
    "decoder/decoder_emb_layer/bias/Adam_1": "decoder/dense/bias/Adam_1",
    "decoder/decoder_emb_layer/kernel/Adam": "decoder/dense/kernel/Adam",
    "decoder/decoder_emb_layer/kernel/Adam_1": "decoder/dense/kernel/Adam_1",
}

saver_variables = tf.global_variables()
var_list = {var.op.name: var for var in saver_variables}

for it in var_map:
    var_list[it] = var_list[var_map[it]]
    del var_list[var_map[it]]
            
saver = tf.train.Saver(var_list=var_list)
for var in var_list:
    print("%s\t%s" % (var, str(var_list[var])))

Variable	<tf.Variable 'Variable:0' shape=() dtype=int32_ref>
stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/basic_lstm_cell/kernel	<tf.Variable 'stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/basic_lstm_cell/kernel:0' shape=(440, 1280) dtype=float32_ref>
stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/basic_lstm_cell/bias	<tf.Variable 'stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/basic_lstm_cell/bias:0' shape=(1280,) dtype=float32_ref>
stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/basic_lstm_cell/kernel	<tf.Variable 'stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/basic_lstm_cell/kernel:0' shape=(440, 1280) dtype=float32_ref>
stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/basic_lstm_cell/bias	<tf.Variable 'stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/basic_lstm_cell/bias:0' shape=(1280,) dtype=float32_ref>
stack_bidirectional_rnn/cell_1/bidirectional_rnn/fw/basic_lstm_cell/kernel	<tf.Variable 'stack_bidirectional_rnn/cell_1/bidirectional_rnn/f

In [3]:
graph = tf.get_default_graph()
decoder_kernel = graph.get_tensor_by_name('decoder/attention_wrapper/basic_lstm_cell/kernel:0')
decoder_kernel_var = tf.get_variable("new_kernel", shape=[1955, 2560])

In [4]:
tf.assign(decoder_kernel_var, tf.pad(decoder_kernel, [[0, 35], [0, 0]]))
print(decoder_kernel_var)
var_list['decoder/attention_wrapper/basic_lstm_cell/kernel'] = decoder_kernel_var

<tf.Variable 'new_kernel:0' shape=(1955, 2560) dtype=float32_ref>


In [5]:
decoder_kernel_adam = graph.get_tensor_by_name('decoder/attention_wrapper/basic_lstm_cell/kernel/Adam:0')
decoder_kernel_var_adam = tf.get_variable("new_kernel_adam", shape=[1955, 2560])
tf.assign(decoder_kernel_var_adam, tf.pad(decoder_kernel_adam, [[0, 35], [0, 0]]))
print(decoder_kernel_var_adam)
var_list['decoder/attention_wrapper/basic_lstm_cell/kernel/Adam'] = decoder_kernel_var_adam

<tf.Variable 'new_kernel_adam:0' shape=(1955, 2560) dtype=float32_ref>


In [6]:
decoder_kernel_adam_1 = graph.get_tensor_by_name('decoder/attention_wrapper/basic_lstm_cell/kernel/Adam_1:0')
decoder_kernel_var_adam_1 = tf.get_variable("new_kernel_adam_1", shape=[1955, 2560])
tf.assign(decoder_kernel_var_adam_1, tf.pad(decoder_kernel_adam_1, [[0, 35], [0, 0]]))
print(decoder_kernel_var_adam_1)
var_list['decoder/attention_wrapper/basic_lstm_cell/kernel/Adam_1'] = decoder_kernel_var_adam_1

<tf.Variable 'new_kernel_adam_1:0' shape=(1955, 2560) dtype=float32_ref>


In [7]:
for var in var_list:
    print("%s\t%s" % (var, str(var_list[var].get_shape())))

Variable	()
stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/basic_lstm_cell/kernel	(440, 1280)
stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/basic_lstm_cell/bias	(1280,)
stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/basic_lstm_cell/kernel	(440, 1280)
stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/basic_lstm_cell/bias	(1280,)
stack_bidirectional_rnn/cell_1/bidirectional_rnn/fw/basic_lstm_cell/kernel	(960, 1280)
stack_bidirectional_rnn/cell_1/bidirectional_rnn/fw/basic_lstm_cell/bias	(1280,)
stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/basic_lstm_cell/kernel	(960, 1280)
stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/basic_lstm_cell/bias	(1280,)
stack_bidirectional_rnn/cell_2/bidirectional_rnn/fw/basic_lstm_cell/kernel	(960, 1280)
stack_bidirectional_rnn/cell_2/bidirectional_rnn/fw/basic_lstm_cell/bias	(1280,)
stack_bidirectional_rnn/cell_2/bidirectional_rnn/bw/basic_lstm_cell/kernel	(960, 1280)
stack_bidirectional_rnn/cell_2/bidirectional_rnn/bw/basic_lst

In [10]:
init_op = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=var_list)
with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, '../../../saved_models/attention_aps_word_context/csp.epoch9-new.ckpt')

In [9]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file(file_name="../../../saved_models/attention_aps_word_context/csp.epoch9-new.ckpt", tensor_name='', all_tensors=False, all_tensor_names=False)

Variable (DT_INT32) []
beta1_power (DT_FLOAT) []
beta2_power (DT_FLOAT) []
decoder/attention_wrapper/attention_layer/kernel (DT_FLOAT) [1280,640]
decoder/attention_wrapper/attention_layer/kernel/Adam (DT_FLOAT) [1280,640]
decoder/attention_wrapper/attention_layer/kernel/Adam_1 (DT_FLOAT) [1280,640]
decoder/attention_wrapper/basic_lstm_cell/bias (DT_FLOAT) [2560]
decoder/attention_wrapper/basic_lstm_cell/bias/Adam (DT_FLOAT) [2560]
decoder/attention_wrapper/basic_lstm_cell/bias/Adam_1 (DT_FLOAT) [2560]
decoder/attention_wrapper/basic_lstm_cell/kernel (DT_FLOAT) [1955,2560]
decoder/attention_wrapper/basic_lstm_cell/kernel/Adam (DT_FLOAT) [1955,2560]
decoder/attention_wrapper/basic_lstm_cell/kernel/Adam_1 (DT_FLOAT) [1955,2560]
decoder/attention_wrapper/custom_attention/attention_v (DT_FLOAT) [640]
decoder/attention_wrapper/custom_attention/attention_v/Adam (DT_FLOAT) [640]
decoder/attention_wrapper/custom_attention/attention_v/Adam_1 (DT_FLOAT) [640]
decoder/attention_wrapper/custom_atte